## Data Reading

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df=spark.read.format('parquet').load('abfss://bronze2026@storagespotifyproject20.dfs.core.windows.net/products')

## Drop column

In [0]:
df=df.drop('_rescued_data')

##Functions

In [0]:
# df.createOrReplaceTempView("products")

In [0]:
%sql
create or replace function databricks_cata.bronze.discount_func(p_price double) returns double
language sql
return p_price * 0.90

In [0]:
df=df.withColumn('discount_price',expr('databricks_cata.bronze.discount_func(price)'))

In [0]:
# %sql
# select product_id,databricks_cata.bronze.discount_func(price) as discount_price
# from
# products

## Data Writing

In [0]:
df.write.format('delta').mode('append').save("abfss://silver2026@storagespotifyproject20.dfs.core.windows.net/Products")

## Creating table in silver layer

In [0]:
%sql
create table if not exists databricks_cata.silver.Products_silver
using delta
location 'abfss://silver2026@storagespotifyproject20.dfs.core.windows.net/Products'
